In [81]:
import pandas as pd
from sqlalchemy import create_engine
import datetime as dt
import numpy as np
# Module used to split string with different delimiters
import re


In [82]:
#EXTRACT
#reading csv files into python
Artist_File_Path='Resources/artistDf.csv'
Billboard_File_Path='Resources/billboardHot100_1999-2019_sample.csv'
Grammy_File_Path='Resources/grammySongs_1999-2019.csv'
#Making them into dataframe
Adf=pd.read_csv(Artist_File_Path)
Bdf = pd.read_csv(Billboard_File_Path)
Gdf=pd.read_csv(Grammy_File_Path)


In [83]:
def login():
    pw=input("Please enter your password for PGAdmin: ")
    return pw

In [84]:
#TRANSFORM
#eliminate extra columns
#Artist Table
Adf=Adf[['Artist','Followers','Genres','NumAlbums','Gender']].copy()
#Billboard Table
Bdf = Bdf[['Artists', 'Name', 'Peak.position', 'Weeks.on.chart','Week','Genre']].copy()
#Grammy Table
Gdf=Gdf[['Artist','GrammyAward','GrammyYear','Name', 'Genre']].copy()

In [85]:
#get rid of all na columns
Adf=Adf.dropna()
Bdf=Bdf.dropna()
Gdf=Gdf.dropna()
#changed all strings to lower case
Gdf = Gdf.astype(str).apply(lambda x: x.str.lower())
Adf = Adf.astype(str).apply(lambda x: x.str.lower())
Bdf = Bdf.astype(str).apply(lambda x: x.str.lower())

In [86]:
# Rename the column headers
Adf = Adf.rename(columns={'Artist':'artist', 
'Genres': 'genre',
'NumAlbums': 'num_albums',
'Gender': 'gender',
'Group.Solo': 'group_solo'})

# Clean the data by dropping duplicates and setting the index
Adf.drop_duplicates('artist', inplace=True)
Adf.set_index('artist', inplace=True)
Adf.head()

,Followers,genre,num_albums,gender
artist,,,,
ed sheeran,52698756,"pop,uk pop",8,m
justin bieber,30711450,"canadian pop,dance pop,pop,post-teen pop",10,m
jonas brothers,3069527,"boy band,dance pop,pop,post-teen pop",10,m
drake,41420478,"canadian hip hop,canadian pop,hip hop,pop rap,...",11,m
chris brown,9676862,"dance pop,pop,pop rap,r&b,rap",6,m


In [87]:
#Transform Week column into year
Bdf['Week'] = pd.to_datetime(Bdf['Week'], format = '%Y-%m-%d').map(lambda x: x.strftime('%Y'))

# Rename the column headers
Bdf = Bdf.rename(columns={'Artists':'artist',
                                                              'Name': 'name_of_song',
                                                              'Peak.position': 'peak_position',
                                                              'Weeks.on.chart': 'weeks_on_chart',
                                                              'Week': 'year',
                                                              'Genre': 'genre'})


#Split artists at (,) and stack them in separate rows   
Bdf = Bdf.drop('artist', axis=1).join(Bdf['artist'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).rename('artist'))

#Keep only first row when there are artist/song duplicate
Bdf.drop_duplicates(['artist', 'name_of_song'], keep='first', inplace=True)
Bdf['artist'].replace('', np.nan, inplace=True)
Bdf = Bdf.dropna()
Bdf.reset_index(drop=True, inplace=True)
Bdf

,name_of_song,peak_position,weeks_on_chart,year,genre,artist
0,old town road,1.0,7.0,2019,"country,atlanta,alternative country,hip-hop,tr...",lil nas
1,bad guy,2.0,13.0,2019,"hip-hop,dark pop,house,trap,memes,alternative ...",billie eilish
2,talk,3.0,20.0,2019,"synth-pop,pop",khalid
3,i don't care,2.0,7.0,2019,"canada,uk,dance,dance-pop,pop",ed sheeran
4,i don't care,2.0,7.0,2019,"canada,uk,dance,dance-pop,pop",justin bieber
...,...,...,...,...,...,...
1894,automatic,35.0,19.0,2014,"rock,country",miranda lambert
1895,wild wild love,30.0,18.0,2014,"pop,rap",pitbull
1896,demons,6.0,61.0,2014,"adult alternative,adult contemporary,alternati...",imagine dragons
1897,beat of the music,44.0,20.0,2014,country,brett eldredge


In [88]:
# Rename the column headers
Gdf = Gdf.rename(columns={'Artist':'artist',
                                                                    'GrammyAward': 'grammy_award',
                                                                    'GrammyYear': 'grammy_year',
                                                                    'Name': 'name_of_song',
                                                                    'Genre': 'genre'})


Gdf = Gdf.reset_index()

del Gdf['index']
#Replacing certain words with an empty space
Gdf['artist'] = Gdf['artist'].str.replace('songwriters','')
Gdf['artist'] = Gdf['artist'].str.replace('songwriter','')
Gdf['artist'] = Gdf['artist'].str.replace('artists','')
Gdf['artist'] = Gdf['artist'].str.replace('artist','')
Gdf['artist'] = Gdf['artist'].str.replace('soloists','')
Gdf['artist'] = Gdf['artist'].str.replace('soloist','')
Gdf['artist'] = Gdf['artist'].str.replace('(','')
Gdf['artist'] = Gdf['artist'].str.replace(')','')
Gdf['artist'] = Gdf['artist'].str.replace(u'\xa0', u'')

#Split artists at (,) and stack them in separate rows   
Gdf  = Gdf .drop('artist', axis=1).join(Gdf ['artist'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).rename('artist'))
Gdf  = Gdf .drop('artist', axis=1).join(Gdf ['artist'].str.split('& |; |, |and |Featuring |with |featuring',expand=True).stack().reset_index(level=1, drop=True).rename('artist'))
Gdf['artist'].replace('', np.nan, inplace=True)
Gdf.reset_index(drop=True, inplace=True)
Gdf = Gdf.drop_duplicates(['grammy_award','name_of_song','artist'], keep='first')
Gdf

,grammy_award,grammy_year,name_of_song,genre,artist
0,record of the year,2018,this is america,general,childish gambino
1,song of the year,2018,this is america,general,childish gambino
2,best pop solo performance,2018,joanne (where do you think you're goin'?),pop,lady gaga
3,best pop duo/group performance,2018,shallow,pop,lady gaga
4,best pop duo/group performance,2018,shallow,pop,bradley cooper
...,...,...,...,...,...
1725,best rap performance by a duo or group,1999,you got me,rap,the roots
1726,best rap performance by a duo or group,1999,you got me,rap,erykah badu
1727,best female country vocal performance,1999,man! i feel like a woman!,country,shania twain
1728,best male country vocal performance,1999,choices,country,george jones


In [94]:
B_Gdf=pd.merge(Bdf,Gdf,on='name_of_song')
B_Gdf=B_Gdf[["name_of_song",	"peak_position",	"weeks_on_chart",	"year",	"genre_y", "artist_x",	"grammy_award",	"grammy_year"]]
B_Gdf=B_Gdf.drop_duplicates()
B_Gdf.groupby(['name_of_song','artist_x',	"peak_position","year",	"weeks_on_chart" ])['grammy_award'].apply(', '.join).reset_index()
B_Gdf.rename(columns={"artist_x":"artist","genre_y":"genre"})

,name_of_song,peak_position,weeks_on_chart,year,genre,artist,grammy_award,grammy_year
0,shallow,1.0,39.0,2019,pop,lady gaga,best pop duo/group performance,2018
2,shallow,1.0,39.0,2019,composing/arranging,lady gaga,best song written for visual media,2018
4,shallow,1.0,39.0,2019,pop,bradley cooper,best pop duo/group performance,2018
6,shallow,1.0,39.0,2019,composing/arranging,bradley cooper,best song written for visual media,2018
8,you say,29.0,43.0,2019,gospel/contemporary christian music,lauren daigle,best contemporary christian music performance/...,2018
...,...,...,...,...,...,...,...,...
156,something in the water,24.0,20.0,2015,country,carrie underwood,best country solo performance,2014
158,work it,2.0,26.0,2015,rap,"missy ""misdemeanor"" elliott",best female rap solo performance,2003
159,get ur freak on,7.0,25.0,2015,rap,"missy ""misdemeanor"" elliott",best rap solo performance,2001
160,rather be,10.0,31.0,2015,dance/electronic music,clean bandit,best dance recording,2014


In [90]:
#LOAD
import psycopg2
pw=login()
connection = None
try:
    # In PostgreSQL, default username is 'postgres' and password is 'postgres'.
    # And also there is a default database exist named as 'postgres'.
    # Default host is 'localhost' or '127.0.0.1'
    # And default port is '54322'.
    connection= psycopg2.connect(
      database='postgres', user='postgres', password=pw, host='127.0.0.1', port= '5432'
    )
    print('Database connected.')

except:
    print('Database not connected.')
#Check is the database exists or not
if connection is not None:
    connection.autocommit = True

    cur = connection.cursor()
    cur.execute("SELECT datname FROM pg_database;")

    list_database = cur.fetchall()
    if ('artist_db',) in list_database:
        print(" Database artist_db already exist")
    else:
        print(" Database artist_db does not exist, creating one right now.......")
        sql = '''CREATE database artist_db'''
        cur.execute(sql)
        print("Database created successfully........")
    connection.close()
    print('Done')


Database connected.
 Database artist_db already exist
Done


In [91]:
#Create connection
rds_connection_string = f"postgres:{pw}@localhost:5432/artist_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [92]:
#Append Dataframs as tables to artist_db
Adf.to_sql(name='artists', con=engine, if_exists='append', index=False)
Bdf.to_sql(name='billboard', con=engine, if_exists='append', index=False)
Gdf.to_sql(name='grammy_songs', con=engine, if_exists='append', index=False)
B_Gdf.to_sql(name='grammy_songs_joined_billboard', con=engine, if_exists='append', index=False)
